In [1]:
%matplotlib inline
import dgl
import copy
import glob
import pprint
import numpy as np
import torch
import torch.nn as nn
import awkward as ak
import networkx as nx
import matplotlib.pyplot as plt
from os import path
from pathlib import Path
from trainresults import TrainResults
from train_eval_func import train, evaluate
from copy import deepcopy
from dgl.data import DGLDataset
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from TauGraphDatasetInfo import TauGraphDatasetInfo
from CustomGCNs import CustomGCN_OnlyNFeatSumMsg, CustomGCN_OnlyNFeatMeanMsg
from TauGraphDataset import TauGraphDataset, GetNodeFeatureVectors, GetEdgeFeatureVectors, GetNeighborNodes, GetEdgeList

plt.rcParams.update({'font.size': 20})
plt.rcParams['text.usetex'] = True
lw = 2
xyLabelFontSize = 20
xLabelPad = 10
yLabelPad = 15
pp = pprint.PrettyPrinter()

Using backend: pytorch


In [2]:
# training
def trainEpochs(model, device, dataloader, optimizer, loss_fn, batchsize, nEpochs):
    results = TrainResults()
    results.startTrainingTimer()
    bestModel = None
    bestValAcc = 0.0

    for epoch in range(nEpochs):
        # train
        epochLoss = train(model, device, dataloader, optimizer, loss_fn, batchsize, results)

        # evaluate
        train_result = evaluate(model, device, train_dataloader, loss_fn)
        val_result = evaluate(model, device, val_dataloader, loss_fn)
        test_result = evaluate(model, device, test_dataloader, loss_fn)

        results.addEpochResult(epochLoss, train_result, val_result, test_result)
        results.printLastResult()

        if results.best_val_acc > bestValAcc:
            bestValAcc = results.best_val_acc
            bestModel = copy.deepcopy(model)
    
    results.endTrainingTimer()

    return results, bestModel

In [3]:
def getDatasetNames(datasetDir):
    files = glob.glob(datasetDir + '/*.json', recursive=True)
    files.sort()
    datasetDirectories = [path.dirname(file) for file in files]
    datasetnames = [path.normpath(dir).split(path.sep)[-1] for dir in datasetDirectories]
    return datasetDirectories, datasetnames

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')

datasetDir = '/ceph/aissac/ntuple_for_graphs/prod_2018_v2_processed_v5_THESIS/trimmed_200000_and_cut_puppiWeightNoLep_greater_0_and_deltaR_smaller_0point5/Graphs_DYJetsToLL_M-50_genuineTaus_and_jets'
datasetDirs, datasetNames = getDatasetNames(datasetDir)
print(datasetDirs)
print(datasetNames)

device: cpu
['/ceph/aissac/ntuple_for_graphs/prod_2018_v2_processed_v5_THESIS/trimmed_200000_and_cut_puppiWeightNoLep_greater_0_and_deltaR_smaller_0point5/Graphs_DYJetsToLL_M-50_genuineTaus_and_jets']
['Graphs_DYJetsToLL_M-50_genuineTaus_and_jets']


datasetName = datasetNames[0]
datasetDir = datasetDirs[0]
dataset = TauGraphDataset(datasetName, datasetDir)
dataset.printProperties()

graph, label = dataset[0]
print(graph)
print(f'Label: {label}')
print(GetNodeFeatureVectors(graph))

In [ ]:
import time
now = time.time()

paramList = [(16, 1, 0), # node_hiddenfeats, #num_messagepasses, #dropout
          (16, 1, 0.3),
          (16, 1, 0.5),
          (32, 1, 0),
          (32, 1, 0.3),
          (32, 1, 0.5),
          (64, 1, 0),
          (64, 1, 0.3),
          (64, 1, 0.5),
          (16, 2, 0), 
          (16, 2, 0.3),
          (16, 2, 0.5),
          (32, 2, 0),
          (32, 2, 0.3),
          (32, 2, 0.5),
          (64, 2, 0),
          (64, 2, 0.3),
          (64, 2, 0.5),
          (16, 3, 0), 
          (16, 3, 0.3),
          (16, 3, 0.5),
          (32, 3, 0),
          (32, 3, 0.3),
          (32, 3, 0.5),
          (64, 3, 0),
          (64, 3, 0.3),
          (64, 3, 0.5),
          (16, 5, 0), 
          (16, 5, 0.3),
          (16, 5, 0.5),
          (32, 5, 0),
          (32, 5, 0.3),
          (32, 5, 0.5),
          (64, 5, 0),
          (64, 5, 0.3),
          (64, 5, 0.5)]

batchSize = 1024
print(f'Device: {device}')

for n_hidden, msgpasses, dropout in paramList:
    for i in range(len(datasetDirs)):
        dataset = TauGraphDataset(datasetNames[i], datasetDirs[i])
        splitIndices = dataset.get_split_indices()

        train_sampler = SubsetRandomSampler(splitIndices['train'])
        val_sampler = SubsetRandomSampler(splitIndices['valid'])
        test_sampler = SubsetRandomSampler(splitIndices['test'])

        train_dataloader = GraphDataLoader(dataset, sampler=train_sampler, batch_size=batchSize, drop_last=False)
        val_dataloader = GraphDataLoader(dataset, sampler=val_sampler, batch_size=batchSize, drop_last=False)
        test_dataloader = GraphDataLoader(dataset, sampler=test_sampler, batch_size=batchSize, drop_last=False)

        # Create the model with given dimensions
        model = CustomGCN_OnlyNFeatSumMsg(dataset.dim_nfeats, n_hidden, dataset.num_graph_classes, msgpasses, dropout).to(device)
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        loss_fn = nn.CrossEntropyLoss()
        epochs = 30

        # train
        print(f'Beginning training on dataset {datasetNames[i]}')
        results, bestmodel = trainEpochs(model, device, train_dataloader, optimizer, loss_fn, batchSize, epochs)
        results.printBestResult()

        # save results
        outputFolder = path.join(datasetDirs[i], f'Output_CustomGCN_OnlyNFeatSumMsg_NHiddenFeat_{n_hidden}_MsgPasses_{msgpasses}_Dropout_{dropout}')
        Path(outputFolder).mkdir(parents=True, exist_ok=True)

        results.savePlots(outputFolder)
        results.dumpSummary(outputFolder)
        results.pickledump(outputFolder)

        # save the best model for inference. (when loading for inference -> model.eval()!! )
        # https://pytorch.org/tutorials/beginner/saving_loading_models.html#what-is-a-state-dict
        torch.save(bestmodel.state_dict(), path.join(outputFolder, 'model.pt'))

        print(f'------------------({i+1}/{len(datasetDirs)}) models trained------------------\n')


end = time.time()
elapsed = end - now
print(f'{elapsed} seconds elapsed')

Device: cpu
Done loading data from cached files.
Beginning training on dataset Graphs_DYJetsToLL_M-50_genuineTaus_and_jets
Epoch: 0, Loss: 0.6889, Validation Loss: 0.5552, Train: 0.632, Validation: 0.631, Test: 0.632, AUC: 0.852
Epoch: 1, Loss: 0.5119, Validation Loss: 0.4721, Train: 0.815, Validation: 0.814, Test: 0.819, AUC: 0.916
Epoch: 2, Loss: 0.4408, Validation Loss: 0.4079, Train: 0.866, Validation: 0.866, Test: 0.871, AUC: 0.938
Epoch: 3, Loss: 0.3799, Validation Loss: 0.3502, Train: 0.883, Validation: 0.883, Test: 0.887, AUC: 0.945
Epoch: 4, Loss: 0.3272, Validation Loss: 0.3056, Train: 0.893, Validation: 0.891, Test: 0.896, AUC: 0.952
Epoch: 5, Loss: 0.2949, Validation Loss: 0.2855, Train: 0.896, Validation: 0.895, Test: 0.899, AUC: 0.955
Epoch: 6, Loss: 0.2786, Validation Loss: 0.2743, Train: 0.896, Validation: 0.895, Test: 0.899, AUC: 0.955
Epoch: 7, Loss: 0.2701, Validation Loss: 0.2655, Train: 0.897, Validation: 0.897, Test: 0.899, AUC: 0.956
Epoch: 8, Loss: 0.2655, Valid

Epoch: 10, Loss: 0.2614, Validation Loss: 0.2593, Train: 0.899, Validation: 0.898, Test: 0.901, AUC: 0.957
Epoch: 11, Loss: 0.2607, Validation Loss: 0.2587, Train: 0.899, Validation: 0.899, Test: 0.901, AUC: 0.957
Epoch: 12, Loss: 0.2602, Validation Loss: 0.2608, Train: 0.899, Validation: 0.899, Test: 0.901, AUC: 0.957
Epoch: 13, Loss: 0.2595, Validation Loss: 0.2613, Train: 0.897, Validation: 0.896, Test: 0.899, AUC: 0.957
Epoch: 14, Loss: 0.2592, Validation Loss: 0.2594, Train: 0.898, Validation: 0.897, Test: 0.899, AUC: 0.957
Epoch: 15, Loss: 0.2583, Validation Loss: 0.2560, Train: 0.899, Validation: 0.899, Test: 0.901, AUC: 0.957
Epoch: 16, Loss: 0.2583, Validation Loss: 0.2591, Train: 0.898, Validation: 0.896, Test: 0.899, AUC: 0.957
Epoch: 17, Loss: 0.2578, Validation Loss: 0.2600, Train: 0.900, Validation: 0.899, Test: 0.902, AUC: 0.957
Epoch: 18, Loss: 0.2581, Validation Loss: 0.2590, Train: 0.898, Validation: 0.898, Test: 0.901, AUC: 0.957
Epoch: 19, Loss: 0.2577, Validation L

In [ ]:
import time
now = time.time()

paramList = [(16, 1, 0), # node_hiddenfeats, #num_messagepasses, #dropout
          (16, 1, 0.3),
          (16, 1, 0.5),
          (32, 1, 0),
          (32, 1, 0.3),
          (32, 1, 0.5),
          (64, 1, 0),
          (64, 1, 0.3),
          (64, 1, 0.5),
          (16, 2, 0), 
          (16, 2, 0.3),
          (16, 2, 0.5),
          (32, 2, 0),
          (32, 2, 0.3),
          (32, 2, 0.5),
          (64, 2, 0),
          (64, 2, 0.3),
          (64, 2, 0.5),
          (16, 3, 0), 
          (16, 3, 0.3),
          (16, 3, 0.5),
          (32, 3, 0),
          (32, 3, 0.3),
          (32, 3, 0.5),
          (64, 3, 0),
          (64, 3, 0.3),
          (64, 3, 0.5),
          (16, 5, 0), 
          (16, 5, 0.3),
          (16, 5, 0.5),
          (32, 5, 0),
          (32, 5, 0.3),
          (32, 5, 0.5),
          (64, 5, 0),
          (64, 5, 0.3),
          (64, 5, 0.5)]

batchSize = 1024
print(f'Device: {device}')

for n_hidden, msgpasses, dropout in paramList:
    for i in range(len(datasetDirs)):
        dataset = TauGraphDataset(datasetNames[i], datasetDirs[i])
        splitIndices = dataset.get_split_indices()

        train_sampler = SubsetRandomSampler(splitIndices['train'])
        val_sampler = SubsetRandomSampler(splitIndices['valid'])
        test_sampler = SubsetRandomSampler(splitIndices['test'])

        train_dataloader = GraphDataLoader(dataset, sampler=train_sampler, batch_size=batchSize, drop_last=False)
        val_dataloader = GraphDataLoader(dataset, sampler=val_sampler, batch_size=batchSize, drop_last=False)
        test_dataloader = GraphDataLoader(dataset, sampler=test_sampler, batch_size=batchSize, drop_last=False)

        # Create the model with given dimensions
        model = CustomGCN_OnlyNFeatMeanMsg(dataset.dim_nfeats, n_hidden, dataset.num_graph_classes, msgpasses, dropout).to(device)
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        loss_fn = nn.CrossEntropyLoss()
        epochs = 30

        # train
        print(f'Beginning training on dataset {datasetNames[i]}')
        results, bestmodel = trainEpochs(model, device, train_dataloader, optimizer, loss_fn, batchSize, epochs)
        results.printBestResult()

        # save results
        outputFolder = path.join(datasetDirs[i], f'Output_CustomGCN_OnlyNFeatMeanMsg_NHiddenFeat_{n_hidden}_MsgPasses_{msgpasses}_Dropout_{dropout}')
        Path(outputFolder).mkdir(parents=True, exist_ok=True)

        results.savePlots(outputFolder)
        results.dumpSummary(outputFolder)
        results.pickledump(outputFolder)

        # save the best model for inference. (when loading for inference -> model.eval()!! )
        # https://pytorch.org/tutorials/beginner/saving_loading_models.html#what-is-a-state-dict
        torch.save(bestmodel.state_dict(), path.join(outputFolder, 'model.pt'))

        print(f'------------------({i+1}/{len(datasetDirs)}) models trained------------------\n')


end = time.time()
elapsed = end - now
print(f'{elapsed} seconds elapsed')